<a href="https://colab.research.google.com/github/HumbertoTiggemann/RAF/blob/main/Extracao_RAF_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Extrator de dados dos __*Relatorio de Atividade Fiscal (RAF)*__ dos Conselhos de Farmácia do Brasil




In [ ]:
pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 76.4 MB/s eta 0:00:00


In [ ]:
pip install xlwt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
pip install openpyxl


In [ ]:
pip install jpype1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 6.6 MB/s eta 0:00:00


In [ ]:
import tabula
import os
import pandas as pd
import numpy as np
import xlwt
import openpyxl
from openpyxl import Workbook
from google.colab import files
import tempfile


In [ ]:
#fazer os uploadas dos arquivos do raf para o drive temporario do Coolab
uploaded = files.upload()

for fn in uploaded.keys():
  print('Arquivo: "{name}", tamanho {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 1_Raf enviado.pdf to 1_Raf enviado.pdf
Saving 1_raf_janeiro_2022 versao 2.pdf to 1_raf_janeiro_2022 versao 2.pdf
Saving 2_Raf enviado.pdf to 2_Raf enviado.pdf
Saving raf - abril - 2022.pdf to raf - abril - 2022.pdf
Saving raf - marco - 2022.pdf to raf - marco - 2022.pdf
Saving Raf enviado (1).pdf to Raf enviado (1).pdf
Saving Raf enviado (2).pdf to Raf enviado (2).pdf
Saving Raf enviado (3).pdf to Raf enviado (3).pdf
Saving RAF enviado (4).pdf to RAF enviado (4).pdf
Saving Raf enviado (5).pdf to Raf enviado (5).pdf
Saving raf enviado 2 (1).pdf to raf enviado 2 (1).pdf
Saving raf enviado 2.pdf to raf enviado 2.pdf
Saving raf enviado.pdf to raf enviado.pdf
Saving RAF Março 2023.pdf to RAF Março 2023.pdf
Saving raf_agosto_2022.pdf to raf_agosto_2022.pdf
Saving raf_dezembro_2022.pdf to raf_dezembro_2022.pdf
Saving raf_fevereiro_2023.pdf to raf_fevereiro_2023.pdf
Saving raf_janeiro_2023.pdf to raf_janeiro_2023.pdf
Saving raf_junho_2022.pdf to raf_junho_2022.pdf
Saving raf_maio_2022.p

In [ ]:
# Define o Conselho Estadual de Farmacia do RAF

crfs_brasil_dic = {
    "CRF-AC": "Conselho Regional de Farmácia do Acre",
    "CRF-AL": "Conselho Regional de Farmácia de Alagoas",
    "CRF-AP": "Conselho Regional de Farmácia do Amapá",
    "CRF-AM": "Conselho Regional de Farmácia do Amazonas",
    "CRF-BA": "Conselho Regional de Farmácia da Bahia",
    "CRF-CE": "Conselho Regional de Farmácia do Ceará",
    "CRF-DF": "Conselho Regional de Farmácia do Distrito Federal",
    "CRF-ES": "Conselho Regional de Farmácia do Espírito Santo",
    "CRF-GO": "Conselho Regional de Farmácia de Goiás",
    "CRF-MA": "Conselho Regional de Farmácia do Maranhão",
    "CRF-MT": "Conselho Regional de Farmácia do Mato Grosso",
    "CRF-MS": "Conselho Regional de Farmácia do Mato Grosso do Sul",
    "CRF-MG": "Conselho Regional de Farmácia de Minas Gerais",
    "CRF-PA": "Conselho Regional de Farmácia do Pará",
    "CRF-PB": "Conselho Regional de Farmácia da Paraíba",
    "CRF-PR": "Conselho Regional de Farmácia do Paraná",
    "CRF-PE": "Conselho Regional de Farmácia de Pernambuco",
    "CRF-PI": "Conselho Regional de Farmácia do Piauí",
    "CRF-RJ": "Conselho Regional de Farmácia do Rio de Janeiro",
    "CRF-RN": "Conselho Regional de Farmácia do Rio Grande do Norte",
    "CRF-RS": "Conselho Regional de Farmácia do Rio Grande do Sul",
    "CRF-RO": "Conselho Regional de Farmácia de Rondônia",
    "CRF-RR": "Conselho Regional de Farmácia de Roraima",
    "CRF-SC": "Conselho Regional de Farmácia de Santa Catarina",
    "CRF-SP": "Conselho Regional de Farmácia de São Paulo",
    "CRF-SE": "Conselho Regional de Farmácia de Sergipe",
    "CRF-TO": "Conselho Regional de Farmácia do Tocantins"
}

sigla = "CRF-" + input("Qual o estado pertence aos arquivos (PDF) do RAF para fazer upload?\n"
                       "Escreva somente a sigla do estado!\n").upper()

while sigla not in crfs_brasil_dic:
    print(f"O estado '{sigla}' não foi encontrado na lista de estados disponíveis.\n")
    sigla = "CRF-" + input("Tente de novo, somente a sigla do estado!\n").upper()

crf_estado = crfs_brasil_dic[sigla]
estado= sigla
print(f"\n   O estado do RAF é {crf_estado}!!\n")
print(f"                       {estado}")

Qual o estado pertence aos arquivos (PDF) do RAF para fazer upload?
Escreva somente a sigla do estado!
rs

   O estado do RAF é Conselho Regional de Farmácia do Rio Grande do Sul!!

                       CRF-RS


In [ ]:
# Defina um novo nome dos arquivos de upload e adiciona em uma lista

lista_pdf = []  #
for i in range(len(uploaded)):
  nome_antigo =list(uploaded.keys())[i]
  novo_nome = "raf_"+estado+"_file_"+str(i)+".pdf"
  os.rename(nome_antigo, novo_nome)# Renomeie o arquivo
  lista_pdf.append(novo_nome)

In [ ]:
#funçao para converter arquivo em csv
def converter_csv(arquivo):
  tabula.convert_into(arquivo, arquivo+"_pg1.csv", output_format="csv", pages='1')
  tabula.convert_into(arquivo, arquivo+"_pg2.csv", output_format="csv", pages='2')
  return arquivo+"_pg1.csv", arquivo+"_pg2.csv"

In [ ]:
#execuçao da função nos arquivos pdf(lista_pdf)
lista_csv= []
for i in range(len(lista_pdf)):
  arquivo= lista_pdf[i]
  arquivo_convertido= converter_csv(arquivo)
  lista_csv.append(arquivo_convertido)
print(lista_csv)

[('raf_CRF-RS_file_0.pdf_pg1.csv', 'raf_CRF-RS_file_0.pdf_pg2.csv'), ('raf_CRF-RS_file_1.pdf_pg1.csv', 'raf_CRF-RS_file_1.pdf_pg2.csv'), ('raf_CRF-RS_file_2.pdf_pg1.csv', 'raf_CRF-RS_file_2.pdf_pg2.csv'), ('raf_CRF-RS_file_3.pdf_pg1.csv', 'raf_CRF-RS_file_3.pdf_pg2.csv'), ('raf_CRF-RS_file_4.pdf_pg1.csv', 'raf_CRF-RS_file_4.pdf_pg2.csv'), ('raf_CRF-RS_file_5.pdf_pg1.csv', 'raf_CRF-RS_file_5.pdf_pg2.csv'), ('raf_CRF-RS_file_6.pdf_pg1.csv', 'raf_CRF-RS_file_6.pdf_pg2.csv'), ('raf_CRF-RS_file_7.pdf_pg1.csv', 'raf_CRF-RS_file_7.pdf_pg2.csv'), ('raf_CRF-RS_file_8.pdf_pg1.csv', 'raf_CRF-RS_file_8.pdf_pg2.csv'), ('raf_CRF-RS_file_9.pdf_pg1.csv', 'raf_CRF-RS_file_9.pdf_pg2.csv'), ('raf_CRF-RS_file_10.pdf_pg1.csv', 'raf_CRF-RS_file_10.pdf_pg2.csv'), ('raf_CRF-RS_file_11.pdf_pg1.csv', 'raf_CRF-RS_file_11.pdf_pg2.csv'), ('raf_CRF-RS_file_12.pdf_pg1.csv', 'raf_CRF-RS_file_12.pdf_pg2.csv'), ('raf_CRF-RS_file_13.pdf_pg1.csv', 'raf_CRF-RS_file_13.pdf_pg2.csv'), ('raf_CRF-RS_file_14.pdf_pg1.csv', 'raf

In [ ]:
#Funçao para criar o dataframe oriundo do csv com o nome de tabela e executa as auteraçoes necessarias
def dataframe(lista):
    tabelas_csv1 = []
    tabelas_csv2 = []

    for i in range(len(lista)):
        tabela_csv1 = pd.read_csv(lista_csv[i][0])
        tabela_csv2 = pd.read_csv(lista_csv[i][1])
        cabecalhos = tabela_csv1.columns # pega os nomes da colunas (cabecarios) e cria as variaveis mês e ano e renomeia a coluna
        data = cabecalhos[4]
        mes= data[8:10]
        ano= data[-4:]
        tabela_csv1.rename(columns={data:'mes_ano'}, inplace=True)
        tabela_csv1["Mês"]= mes #cria coluna Mês com o
        tabela_csv1["Ano"]= ano #cria coluna Ano
        tabela_csv2["Mês"]= mes #cria coluna Mês com o
        tabela_csv2["Ano"]= ano #cria coluna Ano
        novos_nomes = {
                    "Unnamed: 0":"Tipos de Estabelecimentos",
                    'II Informações da Fiscalização': "ESTABELECIMENTO_Regular ESTABELECIMENTO_Irregular ESTABELECIMENTO_TOTAL ESTABELECIMENTO_Sem_Registro" ,
                    'CONSELHO FEDERAL DE FARMÁCIA': "INSPEÇÕES_Assistência_Presente INSPEÇÕES_Assistência_Ausente INSPEÇÕES_Assistência_Outro INSPEÇÕES_TOTAL",
                    'Unnamed: 3': "ATUAÇÃO_Sem_Responsável",
                    'mes_ano':"ATUAÇÃO_Horario_Divergente ATUAÇÃO_Atividade_sem_Farma ATUAÇÃO_Sem_Registro ATUAÇÃO_Farma_Ausente",
                    'Unnamed: 5':"ATUAÇÃO_TOTAL",
                    'Mês':'Mês',
                    'Ano': 'Ano'}#renomeia as colunas principais
        tabela_csv1 = tabela_csv1.rename(columns=novos_nomes)
        colunas_lista = tabela_csv1.columns
        tabela_csv1= tabela_csv1.drop([0,1,2,3,4]).reset_index(drop=True) #apaga as 5 primeiras linhas e reseta o index
        #dividir as colunas conforme o original
        tabela_csv1[["ESTABELECIMENTO_Regular", "ESTABELECIMENTO_Irregular", "ESTABELECIMENTO_TOTAL", "ESTABELECIMENTO_Sem_Registro"]] = tabela_csv1["ESTABELECIMENTO_Regular ESTABELECIMENTO_Irregular ESTABELECIMENTO_TOTAL ESTABELECIMENTO_Sem_Registro"].str.split(' ', expand=True)
        tabela_csv1[["INSPEÇÕES_Assistência_Presente", "INSPEÇÕES_Assistência_Ausente", "INSPEÇÕES_Assistência_Outro", "INSPEÇÕES_TOTAL"]] = tabela_csv1["INSPEÇÕES_Assistência_Presente INSPEÇÕES_Assistência_Ausente INSPEÇÕES_Assistência_Outro INSPEÇÕES_TOTAL"].str.split(' ', expand=True)
        tabela_csv1[["ATUAÇÃO_Horario_Divergente", "ATUAÇÃO_Atividade_sem_Farma", "ATUAÇÃO_Sem_Registro", "ATUAÇÃO_Farma_Ausente"]] = tabela_csv1["ATUAÇÃO_Horario_Divergente ATUAÇÃO_Atividade_sem_Farma ATUAÇÃO_Sem_Registro ATUAÇÃO_Farma_Ausente"].str.split(' ', expand=True)
        # Remover a coluna 'Nome_Idade' original
        tabela_csv1.drop(columns=["ESTABELECIMENTO_Regular ESTABELECIMENTO_Irregular ESTABELECIMENTO_TOTAL ESTABELECIMENTO_Sem_Registro"], inplace=True)
        tabela_csv1.drop(columns=["INSPEÇÕES_Assistência_Presente INSPEÇÕES_Assistência_Ausente INSPEÇÕES_Assistência_Outro INSPEÇÕES_TOTAL"], inplace=True)
        tabela_csv1.drop(columns=["ATUAÇÃO_Horario_Divergente ATUAÇÃO_Atividade_sem_Farma ATUAÇÃO_Sem_Registro ATUAÇÃO_Farma_Ausente"], inplace=True)
        #reordernar as colunas
        tabela_csv2["Mês"]= mes
        tabela_csv2["Ano"]= ano
        cabecalhos_trocados = tabela_csv2.columns.tolist() #valores a serem adicionados na tabela_scv1
        cabecalhos_trocados[-1]= ano
        cabecalhos_trocados[-2]= mes
        tabela_csv1= tabela_csv1[['Tipos de Estabelecimentos','ESTABELECIMENTO_Regular', 'ESTABELECIMENTO_Irregular','ESTABELECIMENTO_TOTAL', 'ESTABELECIMENTO_Sem_Registro','INSPEÇÕES_Assistência_Presente', 'INSPEÇÕES_Assistência_Ausente', 'INSPEÇÕES_Assistência_Outro', 'INSPEÇÕES_TOTAL','ATUAÇÃO_Sem_Responsável','ATUAÇÃO_Horario_Divergente', 'ATUAÇÃO_Atividade_sem_Farma', 'ATUAÇÃO_Sem_Registro', 'ATUAÇÃO_Farma_Ausente','ATUAÇÃO_TOTAL', 'Mês', 'Ano']]
        linha_faltante = {}
        for coluna, cabecalho in zip(tabela_csv1.columns, cabecalhos_trocados):
            linha_faltante[coluna] = cabecalho
        tabela_csv1 = pd.concat([tabela_csv1, pd.DataFrame([linha_faltante])], ignore_index=True)
        tabela_csv1["Estado"] = estado

        tabelas_csv1.append(tabela_csv1)
        tabelas_csv2.append(tabela_csv2)

    return tabelas_csv1, tabelas_csv2

In [ ]:
#executa a funçao do dataframe e cria duas listas de dataframes
tabelas_csv1, tabelas_csv2 = dataframe(lista_csv)


In [ ]:
#ordena a lista do dataframe da pagina1 e renomeia, após faz o donwload dos arquivos um por um
for i in range(len(tabelas_csv1)):
    arquivo_nome = "RAF_ok_"+estado+"_file_"+ str(i) + "_pg1.csv"
    tabelas_csv1[i].to_csv(arquivo_nome, index=False)
    files.download("RAF_ok_"+estado+"_file_"+ str(i) + "_pg1.csv")
# Verifica se o arquivo CSV foi criado
if os.path.exists("RAF_ok_"+estado+"_file_"+ str(i) + "_pg1.csv"):
    print("Os arquivos CSV foram unidos com sucesso!")
else:
    print("Houve um problema ao unir os arquivos CSV.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Os arquivos CSV foram unidos com sucesso!


In [ ]:
# Concatena os dataframes em um único dataframe
dataframe_final = pd.concat(tabelas_csv1)

# Salva o dataframe final como um único arquivo CSV
dataframe_final.to_csv("RAF_ok_"+estado+"_pg1.csv", index=False)

# Verifica se o arquivo CSV foi criado
if os.path.exists("RAF_ok_"+estado+"_pg1.csv"):
    print("Os arquivos CSV foram unidos com sucesso!")
else:
    print("Houve um problema ao unir os arquivos CSV.")


Os arquivos CSV foram unidos com sucesso!
